<a href="https://colab.research.google.com/github/kmk4444/System_engineering/blob/main/Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [ ]:
    "Termodinamik",
    "Akışkanlar mekaniği",
    "Isı transferi",
    "Mekanik tasarım",
    "Robotik",
    "Gömülü sistemler",
    "Yapay zeka",
    "Makine öğrenmesi",
    "Siber güvenlik",
    "Yazılım geliştirme",
    "Elektrik mühendisliği",
    "Telekomünikasyon",
    "Yenilenebilir enerji sistemleri",
    "Kontrol sistemleri",
    "Nanomalzemeler",
    "Biyomalzemeler",
    "Çevre mühendisliği",
    "Hava kirliliği kontrolü",
    "Sürdürülebilirlik",
    "İklim değişikliği",
    "Uçuş dinamiği",
    "Uzay mekaniği",
    "Tıbbi cihazlar",
    "Biyomekanik",
    "Genomik ve proteomik",
    "DevOps",
    "Bulut bilişim",
    "Enerji depolama",
    "Akıllı şebekeler",
    "Elektrikli araçlar",
    "Otonom sürüş sistemleri",
    "İnsansız hava araçları",
    "Jeotermal enerji",
    "Gıda güvenliği",
    "Fermentasyon teknolojisi",
    "Lojistik ve dağıtım",
    "Veri analitiği",
    "Sistem analizi",
    "Kalite kontrol",
    "Proje yönetimi",
    "Yapı mühendisliği",
    "Geoteknik mühendisliği",
    "Ulaştırma mühendisliği",
    "Hidrolik",
    "Yapı malzemeleri",
    "Deprem mühendisliği",
    "İnşaat yönetimi",
    "Köprü ve tünel tasarımı",
    "Altyapı planlaması",
    "Yazılım mimarisi",
    "Veri yapıları ve algoritmalar",
    "Bilgisayar ağları",
    "Veri tabanı sistemleri",
    "Dağıtık sistemler",
    "İnsan-bilgisayar etkileşimi",
    "Büyük veri analitiği",
    "Proses mühendisliği",
    "Kimyasal reaksiyon mühendisliği",
    "Kütle ve ısı transferi",
    "Biyokimya mühendisliği",
    "Polimer mühendisliği",
    "Petrokimya",
    "İlaç mühendisliği",
    "Gıda mühendisliği",
    "Çevresel kimya",
    "İş etüdü ve ergonomi",
    "Üretim planlama ve kontrol",
    "Yöneylem araştırması",
    "Sistem analizi ve tasarımı",
    "Kalite kontrol ve güvenilirlik",
    "Tedarik zinciri yönetimi",
    "Karar verme modelleri",
    "Simülasyon",
    "Havacılık ve uzay mühendisliği",
    "Aerodinamik",
    "İtki sistemleri",
    "Uçak yapıları",
    "Navigasyon ve kontrol",
    "Uydu teknolojileri",
    "Hava trafik kontrolü",
    "Malzeme karakterizasyonu",
    "Metalurji",
    "Polimerler ve kompozit malzemeler",
    "Seramik malzemeler",
    "Akıllı malzemeler",
    "Yüzey mühendisliği",
    "Korozyon ve korunma",
    "Elektronik malzemeler",
    "Su ve atık su arıtma",
    "Katı atık yönetimi",
    "Çevresel etki değerlendirmesi",
    "Yenilenebilir enerji",
    "Sürdürülebilir kalkınma",
    "Hidroloji",
    "Yeşil bina tasarımı",
    "Sondaj teknolojileri",
    "Üretim mühendisliği",
    "Petrol rafinajı",
    "Doğalgaz işleme",
    "Petrol jeolojisi",
    "Biyomedikal mühendisliği",
    "Tıbbi görüntüleme",
    "Rehabilitasyon mühendisliği",
     "Biyosensörler",
    "Doku mühendisliği",
    "Nöro mühendislik",
    "Klinik mühendislik",
    "Yazılım test ve doğrulama",
    "Agile ve Scrum metodolojileri",
    "Nesne yönelimli programlama",

In [2]:
import csv
from openai import OpenAI
import time
import os
import random
import re

# OpenAI API anahtarınızı güvenli bir şekilde kullanmak için ortam değişkenlerini kullanın
client = OpenAI(
    # This is the default and can be omitted
    api_key='sk-proj-',
)

konular = [
    "Yazılım güvenliği",
    "Mobil uygulama geliştirme",
    "Web teknolojileri",
    "Süreç iyileştirme",
    "Nükleer enerji",
    "Enerji politikaları",
    "Termal enerji sistemleri",
    "Hidrojen teknolojileri",
    "Enerji ekonomisi",
    "Karbon yakalama ve depolama",
    "Otomotiv mühendisliği",
    "Motor teknolojileri",
    "Araç dinamiği",
    "Otomotiv elektroniği",
    "Güvenlik sistemleri",
    "Şasi ve süspansiyon tasarımı",
    "Emisyon kontrolü",
    "Yakıt hücreleri",
    "Araç aerodinamiği",
    "Mekatronik",
    "Sensör ve aktüatörler",
    "Endüstriyel otomasyon",
    "Akıllı sistemler",
    "İnsan-robot etkileşimi",
    "Mineraloji",
    "Sismik analiz",
    "Hidrojeoloji",
    "Maden yatakları",
    "Paleontoloji",
    "Jeotermal enerji",
    "Çevresel jeoloji",
    "Yapısal jeoloji",
    "Deniz mühendisliği",
    "Gemi tasarımı",
    "Deniz yapıları",
    "Deniz araçları hidrodinamiği",
    "Denizaltı teknolojileri",
    "Okyanus mühendisliği",
    "Deniz kirliliği kontrolü",
    "Liman ve kıyı mühendisliği",
    "Deniz enerjisi sistemleri",
    "Gemi inşa süreçleri",
    "Deniz navigasyonu",
    "Gıda işleme teknolojileri",
    "Gıda paketleme",
    "Fermentasyon teknolojisi",
    "Gıda kimyası",
    "Gıda mikrobiyolojisi",
    "Ürün geliştirme",
    "Soğuk zincir yönetimi",
    "Tekstil mühendisliği",
    "Lif teknolojisi",
    "Dokuma ve örme teknikleri",
    "Boyama ve terbiye işlemleri",
    "Teknik tekstiller",
    "Tekstil kimyası",
    "Moda tasarımı",
    "Tekstil makineleri",
    "Kumaş analizi",
    "Nonwoven teknolojileri",
    "Metal üretim süreçleri",
    "Isıl işlemler",
    "Alaşımlar ve özellikleri",
    "Toz metalurjisi",
    "Mekanik özellikler ve testler",
    "Metal şekillendirme",
    "Malzeme yorgunluğu",
    "Kaynak teknolojileri",
    "Kompozit malzemeler",
    "Deprem mühendisliği",
    "Köprü mühendisliği",
    "Tünel mühendisliği",
    "Biyomedikal cihazlar",
    "Yenilenebilir enerjinin entegrasyonu",
    "Şebeke stabilitesi",
    "Enerji piyasası analizi",
    "Robotik süreç otomasyonu",
    "Endüstriyel Nesnelerin İnterneti (IoT)",
    "3D baskı teknolojileri",
    "Sürdürülebilir tasarım",
]

soru_kaliplari = [
    "{0} tasarımının temel adımları nelerdir?",
    "{0} uygulamalarında karşılaşılan etik sorunlar nelerdir?",
    "{0} alanında maliyet optimizasyonu nasıl yapılır?",
    "{0} ve {1} teknolojilerinin entegrasyonu hangi avantajları sağlar?",
    "{0} kullanarak nasıl bir proje planlaması yapılır?",
    "{0} süreçlerinde kalite güvencesi nasıl sağlanır?",
    "{0} geliştirme sürecinde hangi yazılım araçları kullanılır?",
    "{0} alanındaki en iyi uygulama örnekleri nelerdir?",
    "{0} ve {1} arasındaki etkileşimi nasıl optimize edebiliriz?",
    "{0} için uygun test yöntemleri nelerdir?",
    "{0} alanında sürdürülebilir uygulamalar nasıl geliştirilir?",
    "{0} verimliliğini artırmak için hangi stratejiler uygulanabilir?",
    "{0} ile {1} arasındaki sinerjiyi nasıl kullanabiliriz?",
    "{0} uygulamalarında riskleri nasıl azaltabiliriz?",
    "{0} projelerinde proje yönetim metodolojileri nasıl uygulanır?",
    "{0} için gereksinim analizi nasıl yapılır?",
    "{0} alanında inovasyonun önemi nedir?",
    "{0} teknolojisinin tarihsel gelişimi nasıldır?",
    "{0} alanında eğitim ve sertifikasyon imkanları nelerdir?",
    "{0} uygulamalarında güvenlik standartları nelerdir?",
    "{0} ve {1} sistemlerinin karşılaştırılması nasıldır?",
    "{0} alanında karar verme süreçleri nasıl yönetilir?",
    "{0} projelerinde paydaş yönetimi nasıl yapılır?",
    "{0} için veri toplama ve analizi nasıl gerçekleştirilir?",
    "{0} alanında siber güvenlik tehditleri nelerdir?",
    "{0} süreçlerinde otomasyonun rolü nedir?",
    "{0} sistemlerinde hata toleransı nasıl sağlanır?",
    "{0} uygulamalarında yapay zeka nasıl kullanılır?",
    "{0} için performans değerlendirme kriterleri nelerdir?",
    "{0} alanında uluslararası standartlar nelerdir?",
    "{0} uygulamalarında çevresel sürdürülebilirlik nasıl sağlanır?",
    "{0} projelerinde kaynak planlaması nasıl yapılır?",
    "{0} için en uygun mimari yaklaşımlar nelerdir?",
    "{0} alanında model tabanlı sistem mühendisliği nasıl uygulanır?",
    "{0} projelerinde değişiklik yönetimi nasıl yapılır?",
    "{0} ve {1} arasındaki veri entegrasyonu nasıl sağlanır?",
    "{0} için kalite yönetim sistemleri nasıl tasarlanır?",
    "{0} alanında kullanılan simülasyon araçları nelerdir?",
    "{0} uygulamalarında insan faktörlerinin önemi nedir?",
    "{0} ve {1} alanlarında ortak kullanılan teknolojiler nelerdir?",
    "{0} süreçlerinde geri bildirim mekanizmaları nasıl çalışır?",
    "{0} için gereksinimlerin izlenebilirliği nasıl sağlanır?",
    "{0} alanında proje başarısını etkileyen faktörler nelerdir?",
    "{0} sistemlerinin ölçeklenebilirliği nasıl artırılır?",
    "{0} projelerinde maliyet tahmini nasıl yapılır?",
    "{0} uygulamalarında kullanılan en iyi test yöntemleri nelerdir?",
    "{0} alanında iş sağlığı ve güvenliği nasıl sağlanır?",
    "{0} ve {1} teknolojilerinin gelecekteki trendleri nelerdir?",
    "{0} için kullanıcı eğitimi ve desteği nasıl verilir?",
    "{0} süreçlerinde dokümantasyonun önemi nedir?",
    "{0} uygulamalarında veri güvenliği nasıl sağlanır?",
    "{0} için risk değerlendirmesi nasıl yapılır?",
    "{0} projelerinde takım çalışmasının önemi nedir?",
    "{0} alanında inovasyon yönetimi nasıl yapılır?",
    "{0} sistemlerinde enerji verimliliği nasıl artırılır?",
    "{0} ve {1} entegrasyonu sırasında karşılaşılan zorluklar nelerdir?",
    "{0} uygulamalarında müşteri gereksinimleri nasıl belirlenir?",
    "{0} alanında etik kuralların önemi nedir?",
    "{0} süreçlerinde kullanılan modelleme dilleri nelerdir?",
    "{0} için en uygun test stratejileri nelerdir?",
    "{0} projelerinde başarı ölçütleri nelerdir?",
    "{0} alanında sürdürülebilirlik metrikleri nasıl belirlenir?",
    "{0} ve {1} arasındaki farklar nelerdir?",
    "{0} uygulamalarında kalite kontrol süreçleri nasıl işler?",
    "{0} alanında standartların rolü nedir?",
    "{0} için en iyi proje yönetim araçları nelerdir?",
    "{0} süreçlerinde değişiklik kontrolü nasıl yapılır?",
    "{0} projelerinde zaman yönetimi nasıl optimize edilir?",
    "{0} alanında kullanıcı deneyimi nasıl iyileştirilir?",
    "{0} uygulamalarında veri bütünlüğü nasıl sağlanır?",
    "{0} ve {1} alanlarında kullanılan ortak protokoller nelerdir?",
    "{0} sistemlerinde ölçeklendirme stratejileri nelerdir?",
    "{0} için gereksinimlerin önceliklendirilmesi nasıl yapılır?",
    "{0} alanında kullanılan en yeni teknolojiler nelerdir?",
    "{0} projelerinde paydaş katılımı nasıl sağlanır?",
    "{0} süreçlerinde maliyet azaltma yöntemleri nelerdir?",
    "{0} uygulamalarında güvenlik açıkları nasıl tespit edilir?",
    "{0} ve {1} entegrasyonu işletmelere ne gibi faydalar sağlar?",
    "{0} alanında inovasyonu engelleyen faktörler nelerdir?",
    "{0} projelerinde kalite ölçümü nasıl yapılır?",
    "{0} için veri görselleştirme teknikleri nelerdir?",
    "{0} uygulamalarında çevik metodolojiler nasıl uygulanır?",
    "{0} alanında sürekli öğrenme ve gelişim nasıl sağlanır?",
    "{0} süreçlerinde kullanılan en iyi raporlama araçları nelerdir?",
    "{0} ve {1} arasındaki sinerji nasıl artırılır?",
    "{0} projelerinde iletişim yönetimi nasıl yapılır?",
    "{0} için en uygun altyapı nasıl seçilir?",
    "{0} alanında kaynak yönetimi nasıl optimize edilir?",
    "{0} uygulamalarında performans sorunları nasıl giderilir?",
    "{0} sistemlerinde modüler tasarımın faydaları nelerdir?",
    "{0} ve {1} alanlarında standartlaşmanın önemi nedir?",
    "{0} süreçlerinde yapay zeka nasıl kullanılabilir?",
    "{0} projelerinde kalite güvencesi nasıl sağlanır?",
    "{0} uygulamalarında kullanıcı geri bildirimi nasıl toplanır?",
    "{0} alanında veri madenciliği teknikleri nasıl uygulanır?",
    "{0} için uygun prototipleme yöntemleri nelerdir?",
    "{0} ve {1} teknolojilerinin karşılaştırılması nasıldır?",
    "{0} süreçlerinde iyileştirme fırsatları nasıl belirlenir?",
    "{0} projelerinde risk iletişimi nasıl yönetilir?",
    "{0} uygulamalarında performans metrikleri nasıl izlenir?",
    "{0} alanında çalışan ekiplerin eğitimi nasıl yapılır?",
    "{0} ve {1} sistemlerinin entegrasyonu için hangi araçlar kullanılır?",
    "{0} süreçlerinde veri analitiğinin rolü nedir?",
    "{0} projelerinde beklenti yönetimi nasıl yapılır?",
    "{0} uygulamalarında otomasyonun avantajları nelerdir?",
    "{0} alanında çevresel riskler nasıl yönetilir?",
    "{0} için kullanılan en iyi uygulama çerçeveleri nelerdir?",
    "{0} süreçlerinde iş akışları nasıl optimize edilir?",
    "{0} projelerinde değişiklik talepleri nasıl değerlendirilir?",
    "{0} ve {1} teknolojilerinin birleşimi hangi yenilikleri getirir?",
    "{0} uygulamalarında uyumluluk nasıl sağlanır?",
    "{0} alanında veri depolama çözümleri nelerdir?",
    "{0} süreçlerinde hata yönetimi nasıl yapılır?",
    "{0} projelerinde karar verme süreçleri nasıl iyileştirilir?",
    "{0} uygulamalarında enerji tüketimi nasıl azaltılır?",
    "{0} ve {1} alanlarında işbirliği nasıl geliştirilir?",
    "{0} için en iyi uygulama standartları nelerdir?",
    "{0} süreçlerinde tedarik zinciri nasıl yönetilir?",
    "{0} projelerinde öğrenilmiş dersler nasıl kaydedilir?",
    "{0} alanında müşteri memnuniyeti nasıl artırılır?",
    "{0} uygulamalarında mobil teknolojilerin rolü nedir?",
    "{0} için en uygun veritabanı çözümleri nelerdir?",
    "{0} süreçlerinde esneklik nasıl sağlanır?",
    "{0} projelerinde kültürel farklılıklar nasıl yönetilir?",
    "{0} alanında veri gizliliği nasıl korunur?",
    "{0} uygulamalarında büyük veri nasıl işlenir?",
    "{0} ve {1} sistemlerinin güvenliği nasıl sağlanır?",
    "{0} süreçlerinde standartlaştırma neden önemlidir?",
    "{0} projelerinde entelektüel mülkiyet hakları nasıl yönetilir?",
    "{0} alanında kullanılan bulut çözümleri nelerdir?",
    "{0} uygulamalarında kullanıcı arayüzü tasarımı nasıl yapılır?",
    "{0} için en iyi eğitim kaynakları nelerdir?",
    "{0} süreçlerinde veri kalitesi nasıl iyileştirilir?",
    "{0} projelerinde bütçe yönetimi nasıl yapılır?",
    "{0} alanında robotik teknolojilerin kullanımı nasıldır?",
    "{0} uygulamalarında esnek çalışma modelleri nasıl uygulanır?",
    "{0} ve {1} teknolojilerinin iş süreçlerine etkisi nedir?",
    "{0} süreçlerinde müşteri ilişkileri nasıl yönetilir?",
    "{0} projelerinde teslimat süresi nasıl optimize edilir?",
    "{0} alanında etik karar verme nasıl desteklenir?",
    "{0} uygulamalarında iş sürekliliği nasıl sağlanır?",
    "{0} için en uygun programlama dilleri nelerdir?",
    "{0} süreçlerinde verimlilik metrikleri nasıl ölçülür?",
    "{0} projelerinde paydaş beklentileri nasıl yönetilir?",
    "{0} alanında yapay zeka etiği nasıl ele alınır?",
    "{0} uygulamalarında kullanıcı katılımı nasıl artırılır?",
    "{0} ve {1} alanlarında veri standardizasyonu nasıl sağlanır?",
    "{0} süreçlerinde sürdürülebilir uygulamalar nasıl teşvik edilir?",
    "{0} projelerinde iletişim kanalları nasıl etkin kullanılır?",
    "{0} için mevcut en iyi uygulama rehberleri nelerdir?",
    "{0} uygulamalarında mobil cihaz güvenliği nasıl sağlanır?",
    "{0} alanında bilgi yönetimi nasıl yapılır?",
    "{0} süreçlerinde çevik yaklaşımlar nasıl uygulanır?",
    "{0} projelerinde entegrasyon stratejileri nelerdir?",
    "{0} uygulamalarında insan-makine etkileşimi nasıl optimize edilir?",
    "{0} ve {1} sistemlerinin birlikte çalışabilirliği nasıl sağlanır?",
    "{0} süreçlerinde sürekli iyileştirme kültürü nasıl oluşturulur?",
    "{0} projelerinde dış kaynak kullanımı nasıl yönetilir?",
    "{0} alanında veri analitiği yetkinlikleri nasıl geliştirilir?",
    "{0} uygulamalarında blockchain teknolojisi nasıl kullanılır?",
    "{0} için kullanılan en yaygın güvenlik protokolleri nelerdir?",
    "{0} süreçlerinde veri entegrasyonu nasıl yapılır?",
    "{0} projelerinde ekip motivasyonu nasıl artırılır?",
    "{0} alanında sürdürülebilir enerji kullanımı nasıl teşvik edilir?",
    "{0} uygulamalarında yapay zeka etik sorunları nelerdir?",
    "{0} ve {1} teknolojilerinin ortak kullanım alanları nelerdir?",
    "{0} süreçlerinde iş akışı yönetimi nasıl iyileştirilir?",
    "{0} projelerinde başarısızlık nedenleri nelerdir?",
    "{0} uygulamalarında çoklu dil desteği nasıl sağlanır?",
    "{0} alanında veri güvenliği tehditleri nelerdir?",
    "{0} süreçlerinde karar destek sistemleri nasıl kullanılır?",
    "{0} projelerinde zaman planlaması nasıl yapılır?",
    "{0} için en iyi uygulama örnekleri nelerdir?",
    "{0} uygulamalarında enerji tasarrufu nasıl sağlanır?",
    "{0} ve {1} alanlarında standartların uyumlaştırılması nasıl yapılır?",
    "{0} süreçlerinde risk önleme stratejileri nelerdir?",
    "{0} projelerinde iletişim engelleri nasıl aşılır?",
    "{0} alanında veri görselleştirme araçları nelerdir?",
    "{0} uygulamalarında kullanıcı deneyimi nasıl geliştirilir?",
    "{0} için kaynak yönetimi nasıl optimize edilir?",
    "{0} süreçlerinde süreç madenciliği nasıl uygulanır?",
    "{0} projelerinde inovasyon nasıl teşvik edilir?",
    "{0} ve {1} teknolojilerinin sinerjisi nasıl kullanılır?",
    "{0} uygulamalarında veri etiketi nasıl yapılır?",
    "{0} alanında teknoloji trendleri nelerdir?",
    "{0} süreçlerinde kalite iyileştirme nasıl sağlanır?",
    "{0} projelerinde başarının anahtar faktörleri nelerdir?",
    "{0} uygulamalarında siber saldırılara karşı korunma yöntemleri nelerdir?",
    "{0} için en uygun iş modelleri nelerdir?",
    "{0} ve {1} sistemlerinin entegrasyonunda dikkat edilmesi gerekenler nelerdir?",
    "{0} süreçlerinde veri gizliliği nasıl korunur?",
    "{0} projelerinde ekip içi iletişim nasıl güçlendirilir?",
    "{0} alanında yapay zeka kullanımı nasıldır?",
    "{0} uygulamalarında sürdürülebilir malzemeler nasıl kullanılır?",
    "{0} için en iyi eğitim programları nelerdir?",
    "{0} süreçlerinde otomasyonun getirdiği zorluklar nelerdir?",
    "{0} projelerinde performans yönetimi nasıl yapılır?",
    "{0} ve {1} alanlarında kullanılan ortak standartlar nelerdir?",
    "{0} uygulamalarında veri sıkıştırma teknikleri nelerdir?",
    "{0} alanında inovasyon kültürü nasıl oluşturulur?",
    "{0} süreçlerinde kullanıcı ihtiyaçları nasıl belirlenir?",
    "{0} projelerinde proje kapsamı nasıl yönetilir?",
    "{0} uygulamalarında veri aktarımı nasıl güvenli hale getirilir?",
    "{0} için stratejik planlama nasıl yapılır?",
    "{0} ve {1} teknolojilerinin birleşimi hangi fırsatları sunar?",
    "{0} süreçlerinde denetim ve kontrol nasıl yapılır?",
    "{0} projelerinde müşteri memnuniyeti nasıl ölçülür?",
    "{0} alanında sürdürülebilirlik hedefleri nasıl belirlenir?",
    "{0} uygulamalarında yapay zeka etik sorunları nelerdir?",
    "{0} için en uygun güvenlik çözümleri nelerdir?",
    "{0} süreçlerinde veri standardizasyonu nasıl sağlanır?",
    "{0} projelerinde iş analizi nasıl yapılır?",
    "{0} uygulamalarında çevresel etkiler nasıl azaltılır?",
    "{0} ve {1} sistemlerinin entegrasyonunda hangi protokoller kullanılır?",
]


def generate_answer(question):
    try:
        response = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[
                {"role": "system", "content": "Sen bir mühendislik uzmanısın ve sorulara detaylı ve anlaşılır cevaplar veriyorsun. Maksimum 500 token cevap verebilirsin. Bu kapsamda cümleleri yarım bıracak şekilde cevap verme."},
                {"role": "user", "content": question}
            ],
            max_tokens=500,
            n=1,
            stop=None,
            temperature=0.7,
        )
        answer = response.choices[0].message.content.strip()
        return answer
    except Exception as e:
        print(f"Bir hata oluştu: {e}")
        return ""

# Cümleler arasındaki boşlukları kaldıran fonksiyon
def remove_spaces_between_sentences(text):
    # Nokta, ünlem veya soru işaretinden sonra gelen boşlukları kaldırır
    return re.sub(r'(?<=[.!?])\s+:', '', text)

with open('dataset.csv', 'w', encoding='utf-8-sig', newline='') as csvfile:
    fieldnames = ['instruction', 'input', 'output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    for konu1 in konular:
        for soru_kalibi in soru_kaliplari:
            if '{1}' in soru_kalibi:
                # İkinci bir konu seçiyoruz
                konu2 = random.choice(konular)
                while konu2 == konu1:
                    konu2 = random.choice(konular)
                instruction = soru_kalibi.format(konu1, konu2)
            else:
                instruction = soru_kalibi.format(konu1)
            output = generate_answer(instruction)
            # Cevabı işliyoruz
            output = remove_spaces_between_sentences(output)
            writer.writerow({'instruction': instruction, 'input': '', 'output': output})
            time.sleep(1)  # API hız sınırlarını aşmamak için bekleme süresi ekleyin

print("Veri seti oluşturuldu.")


KeyboardInterrupt: 